In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )

In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
R_quat, R_x = tg.rsample(R_mu, sigma)

sigma:  tensor([[1., 1., 1.]])
torch.Size([1, 3, 3])


In [7]:
torch.bmm(torch.transpose(R_x, 1, 2), R_x)

tensor([[[ 1.0000e+00, -2.9802e-08,  2.9802e-08],
         [-2.9802e-08,  1.0000e+00,  0.0000e+00],
         [ 2.9802e-08,  0.0000e+00,  1.0000e+00]]])

# Test torch_tsgaussian normal_term

In [8]:
sigma = torch.ones(3).reshape((1,3))
sigma

tensor([[1., 1., 1.]])

In [9]:
tg.normal_term(sigma)

tensor([15.7496])

# Test torch_tsgaussian log_map

In [10]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [11]:
tg.log_map(R_1, R_2)

tensor([0., 0., 0.])

# Test torch_tsgaussian log_probs

In [12]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [13]:
# tg.log_probs(R_x, R_mu, sigma)

In [14]:
np.e ** (-2.7568)

0.06349462641817973

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [15]:
tsg = TSG(None)

In [16]:
print(tsg.distribution)

In [17]:
tsg

In [18]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

sigma:  tensor([[1., 1., 1.]])
torch.Size([1, 3, 3])


(tensor([[0.0045, 0.3181, 0.4641, 0.8267]]),
 tensor([[[ 0.3668, -0.7645,  0.5300],
          [ 0.7702,  0.5691,  0.2879],
          [-0.5218,  0.3026,  0.7976]]]))

# Test TangentSpaceGaussian log_prob_from_params

In [19]:
torch.eye(3).repeat(2,1,1).size()

torch.Size([2, 3, 3])

In [20]:
torch.ones(3).repeat(2,1).size()

torch.Size([2, 3])

In [21]:
# tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

In [22]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

tensor([[[ 4.3462e-01, -4.4829e-01,  4.6658e-01,  1.3235e-01],
         [-1.0090e+00,  4.3349e-01, -9.0677e-02,  4.8691e-01],
         [ 4.2520e-01,  8.2488e-01,  8.2561e-01,  2.2446e-01],
         [-4.4444e+00,  3.5843e-01,  5.4308e-01,  6.2832e-03]],

        [[-3.2731e-01, -2.7019e-01, -4.1250e-01, -2.9608e-01],
         [ 1.5400e+00,  8.9806e-01, -1.3873e+00,  5.4434e-01],
         [ 3.5008e-03,  4.9183e-01, -5.8710e-01, -1.0904e-01],
         [-3.3795e-01,  4.5905e-01, -7.1282e-01,  1.8377e-01]]])

Again, codes can run, but need to check correctness.

# Try to run training

In [23]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [24]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 0.0,
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 500, eval_freq = 100, n_eval_episodes = 100)

In [ ]:
from torch import autograd
with autograd.detect_anomaly():
    main(None)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.97    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 206      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.12    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 322      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.93    |
| time/              |          |
|    episodes        | 12       |
|    fps             |

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.14    |
| time/              |          |
|    episodes        | 100      |
|    fps             | 796      |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/var/folders/kr/ks3dcr657632lx8hgb0jfh_w0000gn/T/ipykernel_10746/381243426.py:2: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():
/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packages/stable_baselines3/common/buffers.py:220: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 4.82GB > 0.54GB
  warnings.warn(
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


tensor([[-0.0137,  0.0220, -0.0723,  0.0380, -0.0467,  0.0068, -0.0937, -0.0538,
          0.0372, -0.0324,  0.0119,  0.0232]])
sigma:  tensor([[0.0137, 0.0220, 0.0723]])
torch.Size([1, 3, 3])
tensor([[-0.0135,  0.0220, -0.0723,  ..., -0.0324,  0.0117,  0.0232],
        [-0.0133,  0.0218, -0.0727,  ..., -0.0327,  0.0115,  0.0227],
        [-0.0136,  0.0220, -0.0729,  ..., -0.0324,  0.0113,  0.0225],
        ...,
        [-0.0137,  0.0210, -0.0727,  ..., -0.0326,  0.0116,  0.0230],
        [-0.0136,  0.0220, -0.0726,  ..., -0.0327,  0.0116,  0.0227],
        [-0.0135,  0.0216, -0.0728,  ..., -0.0326,  0.0114,  0.0226]],
       grad_fn=<AddmmBackward0>)
sigma:  tensor([[0.0135, 0.0220, 0.0723],
        [0.0133, 0.0218, 0.0727],
        [0.0136, 0.0220, 0.0729],
        [0.0134, 0.0220, 0.0726],
        [0.0136, 0.0220, 0.0729],
        [0.0136, 0.0221, 0.0729],
        [0.0139, 0.0213, 0.0728],
        [0.0139, 0.0214, 0.0726],
        [0.0137, 0.0211, 0.0727],
        [0.0138, 0.0214, 0

tensor([[-0.0135,  0.0220, -0.0723,  ..., -0.0324,  0.0117,  0.0232],
        [-0.0133,  0.0218, -0.0727,  ..., -0.0327,  0.0115,  0.0227],
        [-0.0136,  0.0220, -0.0729,  ..., -0.0324,  0.0113,  0.0225],
        ...,
        [-0.0137,  0.0210, -0.0727,  ..., -0.0326,  0.0116,  0.0230],
        [-0.0136,  0.0220, -0.0726,  ..., -0.0327,  0.0116,  0.0227],
        [-0.0135,  0.0216, -0.0728,  ..., -0.0326,  0.0114,  0.0226]])
sigma:  tensor([[0.0135, 0.0220, 0.0723],
        [0.0133, 0.0218, 0.0727],
        [0.0136, 0.0220, 0.0729],
        [0.0134, 0.0220, 0.0726],
        [0.0136, 0.0220, 0.0729],
        [0.0136, 0.0221, 0.0729],
        [0.0139, 0.0213, 0.0728],
        [0.0139, 0.0214, 0.0726],
        [0.0137, 0.0211, 0.0727],
        [0.0138, 0.0214, 0.0727],
        [0.0139, 0.0213, 0.0728],
        [0.0135, 0.0216, 0.0728],
        [0.0136, 0.0222, 0.0727],
        [0.0137, 0.0215, 0.0724],
        [0.0135, 0.0220, 0.0722],
        [0.0133, 0.0219, 0.0725],
        [0.013

...

tensor([[-0.0138,  0.0214, -0.0728,  ..., -0.0324,  0.0118,  0.0230],
        [-0.0133,  0.0218, -0.0725,  ..., -0.0330,  0.0115,  0.0229],
        [-0.0133,  0.0216, -0.0726,  ..., -0.0326,  0.0115,  0.0229],
        ...,
        [-0.0134,  0.0215, -0.0724,  ..., -0.0329,  0.0115,  0.0231],
        [-0.0138,  0.0213, -0.0725,  ..., -0.0323,  0.0117,  0.0233],
        [-0.0137,  0.0208, -0.0726,  ..., -0.0326,  0.0118,  0.0234]])
sigma:  tensor([[0.0138, 0.0214, 0.0728],
        [0.0133, 0.0218, 0.0725],
        [0.0133, 0.0216, 0.0726],
        [0.0134, 0.0217, 0.0727],
        [0.0138, 0.0217, 0.0729],
        [0.0137, 0.0213, 0.0727],
        [0.0138, 0.0212, 0.0725],
        [0.0139, 0.0213, 0.0728],
        [0.0135, 0.0220, 0.0723],
        [0.0137, 0.0221, 0.0725],
        [0.0137, 0.0217, 0.0725],
        [0.0135, 0.0214, 0.0724],
        [0.0136, 0.0213, 0.0726],
        [0.0134, 0.0219, 0.0728],
        [0.0137, 0.0222, 0.0723],
        [0.0138, 0.0221, 0.0730],
        [0.013

tensor([[-0.0138,  0.0221, -0.0728,  0.0382, -0.0470,  0.0071, -0.0940, -0.0540,
          0.0381, -0.0327,  0.0113,  0.0225]])
sigma:  tensor([[0.0138, 0.0221, 0.0728]])
torch.Size([1, 3, 3])
tensor([[-0.0134,  0.0219, -0.0728,  ..., -0.0324,  0.0116,  0.0226],
        [-0.0137,  0.0218, -0.0727,  ..., -0.0324,  0.0118,  0.0229],
        [-0.0137,  0.0215, -0.0725,  ..., -0.0326,  0.0116,  0.0231],
        ...,
        [-0.0133,  0.0218, -0.0727,  ..., -0.0327,  0.0115,  0.0227],
        [-0.0135,  0.0219, -0.0726,  ..., -0.0323,  0.0118,  0.0229],
        [-0.0135,  0.0219, -0.0726,  ..., -0.0323,  0.0118,  0.0229]],
       grad_fn=<AddmmBackward0>)
sigma:  tensor([[0.0134, 0.0219, 0.0728],
        [0.0137, 0.0218, 0.0727],
        [0.0137, 0.0215, 0.0725],
        [0.0134, 0.0219, 0.0725],
        [0.0135, 0.0213, 0.0724],
        [0.0139, 0.0213, 0.0728],
        [0.0135, 0.0219, 0.0726],
        [0.0137, 0.0214, 0.0729],
        [0.0138, 0.0218, 0.0731],
        [0.0133, 0.0217, 0

tensor([[-0.0134,  0.0219, -0.0728,  ..., -0.0324,  0.0116,  0.0226],
        [-0.0137,  0.0218, -0.0727,  ..., -0.0324,  0.0118,  0.0229],
        [-0.0137,  0.0215, -0.0725,  ..., -0.0326,  0.0116,  0.0231],
        ...,
        [-0.0133,  0.0218, -0.0727,  ..., -0.0327,  0.0115,  0.0227],
        [-0.0135,  0.0219, -0.0726,  ..., -0.0323,  0.0118,  0.0229],
        [-0.0135,  0.0219, -0.0726,  ..., -0.0323,  0.0118,  0.0229]])
sigma:  tensor([[0.0134, 0.0219, 0.0728],
        [0.0137, 0.0218, 0.0727],
        [0.0137, 0.0215, 0.0725],
        [0.0134, 0.0219, 0.0725],
        [0.0135, 0.0213, 0.0724],
        [0.0139, 0.0213, 0.0728],
        [0.0135, 0.0219, 0.0726],
        [0.0137, 0.0214, 0.0729],
        [0.0138, 0.0218, 0.0731],
        [0.0133, 0.0217, 0.0725],
        [0.0136, 0.0214, 0.0725],
        [0.0135, 0.0219, 0.0728],
        [0.0138, 0.0215, 0.0727],
        [0.0138, 0.0210, 0.0727],
        [0.0138, 0.0218, 0.0731],
        [0.0137, 0.0215, 0.0724],
        [0.013

tensor([[-0.0133,  0.0219, -0.0724,  0.0381, -0.0468,  0.0064, -0.0936, -0.0538,
          0.0379, -0.0330,  0.0115,  0.0229]])
sigma:  tensor([[0.0133, 0.0219, 0.0724]])
torch.Size([1, 3, 3])
tensor([[-0.0138,  0.0217, -0.0728,  ..., -0.0326,  0.0112,  0.0228],
        [-0.0139,  0.0212, -0.0725,  ..., -0.0324,  0.0116,  0.0232],
        [-0.0138,  0.0219, -0.0726,  ..., -0.0327,  0.0114,  0.0228],
        ...,
        [-0.0134,  0.0219, -0.0725,  ..., -0.0327,  0.0116,  0.0227],
        [-0.0139,  0.0211, -0.0726,  ..., -0.0323,  0.0116,  0.0231],
        [-0.0140,  0.0211, -0.0728,  ..., -0.0322,  0.0120,  0.0231]],
       grad_fn=<AddmmBackward0>)
sigma:  tensor([[0.0138, 0.0217, 0.0728],
        [0.0139, 0.0212, 0.0725],
        [0.0138, 0.0219, 0.0726],
        [0.0137, 0.0220, 0.0726],
        [0.0137, 0.0220, 0.0723],
        [0.0138, 0.0210, 0.0727],
        [0.0138, 0.0217, 0.0729],
        [0.0133, 0.0218, 0.0725],
        [0.0136, 0.0217, 0.0723],
        [0.0138, 0.0221, 0

tensor([[-0.0138,  0.0217, -0.0728,  ..., -0.0326,  0.0112,  0.0228],
        [-0.0139,  0.0212, -0.0725,  ..., -0.0324,  0.0116,  0.0232],
        [-0.0138,  0.0219, -0.0726,  ..., -0.0327,  0.0114,  0.0228],
        ...,
        [-0.0134,  0.0219, -0.0725,  ..., -0.0327,  0.0116,  0.0227],
        [-0.0139,  0.0211, -0.0726,  ..., -0.0323,  0.0116,  0.0231],
        [-0.0140,  0.0211, -0.0728,  ..., -0.0322,  0.0120,  0.0231]])
sigma:  tensor([[0.0138, 0.0217, 0.0728],
        [0.0139, 0.0212, 0.0725],
        [0.0138, 0.0219, 0.0726],
        [0.0137, 0.0220, 0.0726],
        [0.0137, 0.0220, 0.0723],
        [0.0138, 0.0210, 0.0727],
        [0.0138, 0.0217, 0.0729],
        [0.0133, 0.0218, 0.0725],
        [0.0136, 0.0217, 0.0723],
        [0.0138, 0.0221, 0.0729],
        [0.0138, 0.0221, 0.0728],
        [0.0138, 0.0217, 0.0726],
        [0.0138, 0.0213, 0.0728],
        [0.0138, 0.0213, 0.0728],
        [0.0134, 0.0219, 0.0724],
        [0.0138, 0.0212, 0.0725],
        [0.013

In [ ]:
%load_ext tensorboard
import tensorflow as tf
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

In [ ]:
%tensorboard --logdir ./sac

# Experiments for batch operations

In [ ]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

In [ ]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [ ]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)

# Question to ask: the original wahba problem action is (4,), in our case actions are (3,3).